In [ ]:
pip install opencv-python transformers torch sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 10.5 MB/s eta 0:00:00


In [ ]:
!pip install opencv-python

In [ ]:
import os
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from sentence_transformers import SentenceTransformer, util
import numpy as np
from PIL import Image

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [ ]:
sbert = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def sample_frames_uniformly(video_path, num_samples=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_indices = np.linspace(0, total_frames - 1, num_samples, dtype=int)

    frames = []
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    for i in range(total_frames):
        success, frame = cap.read()
        if not success:
            break
        if i in sample_indices:
            frames.append((i, frame))
    cap.release()
    return frames, fps, (width, height)

def generate_caption(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

def compare_query_to_captions(captions, query):
    query_embedding = sbert.encode(query, convert_to_tensor=True)
    caption_embeddings = sbert.encode(captions, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, caption_embeddings)
    return similarities[0].cpu().numpy()

def save_frames_to_directory(frames, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for idx, (frame_idx, frame) in enumerate(frames):
        frame_path = os.path.join(output_dir, f"frame_{frame_idx}.jpg")
        cv2.imwrite(frame_path, frame)

def main(video_path, num_samples=30, top_k=5, query="", output_dir="matched_frames"):
    print(f"[INFO] Using prompt: {query}")

    print("[INFO] Sampling frames...")
    frames, fps, frame_size = sample_frames_uniformly(video_path, num_samples)

    print("[INFO] Generating captions...")
    captions = []
    for i, frame in frames:
        caption = generate_caption(frame)
        print(f"Frame {i}: {caption}")
        captions.append(caption)

    print("[INFO] Matching captions with query...")
    similarities = compare_query_to_captions(captions, query)

    top_k_indices = np.argsort(similarities)[-top_k:][::-1]
    top_frames = [frames[i] for i in top_k_indices]

    print("\nTop Matching Captions with Timestamps:")
    for idx in top_k_indices:
        frame_idx = frames[idx][0]
        timestamp_sec = frame_idx / fps
        minutes = int(timestamp_sec // 60)
        seconds = int(timestamp_sec % 60)
        timestamp_str = f"{minutes:02d}:{seconds:02d}"
        print(f"Frame {frame_idx} [{timestamp_str}]: '{captions[idx]}' | Similarity: {similarities[idx]:.4f}")

    save_frames_to_directory(top_frames, output_dir)
    print(f"Saved {len(top_frames)} matching frames in directory: {output_dir}")

if __name__ == "__main__":
    video_path = "/content/videoplayback (2).mp4"
    main(video_path, num_samples=50, top_k=5, query="eggs are being cooked in a pan", output_dir="matched_frames")

[INFO] Using prompt: eggs are being cooked in a pan
[INFO] Sampling frames...
[INFO] Generating captions...
Frame 0: there are many ducks in a bin with a yellow box
Frame 1216: there is a man walking up the stairs in a house
Frame 2433: there are two ducks in a plastic container with a yellow box
Frame 3649: there is a man standing in a barn with a cat
Frame 4866: there is a man that is using a hose to water a chicken coop
Frame 6082: there is a chicken that is eating out of a bowl
Frame 7299: there is a man walking in a field with sheep in the fog
Frame 8515: there is a man in rubber boots and rubber boots feeding geese
Frame 9732: there are many ducks walking in the grass together
Frame 10948: ducks are eating out of a bowl in the grass
Frame 12165: araffe walking through a field of tall grass in the fog
Frame 13381: there are many ducks that are standing in a field
Frame 14598: there is a man riding a bike in a field with a dog
Frame 15814: there is a man that is standing in the gra